## Load dependencies

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Flatten, Dense, Dropout, Activation, Input
from keras.optimizers import Adam
from keras import regularizers
from keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping
import pickle
from ourUtils import *
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

### Initialize parameters

In [ ]:
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.8
set_session(tf.Session(config=config))

### Define network

In [ ]:
# Load the convolutional part of the VGG16 network 
vggConv = VGG16(weights='imagenet', include_top=False)

# Input to network
vggInput = Input(shape=(224, 224, 3), name='image_input')
# Output of convolutional part
output_vggConv = vggConv(vggInput)
# Label predictive layers. Initialized using glorot (Xavier's), L2 regularization and dropout
lpmF = Flatten()(output_vggConv)
lpm1 = Dense(2048, activation='relu', kernel_initializer='glorot_normal', bias_initializer='glorot_normal',
            kernel_regularizer=regularizers.l2(0.01))(lpmF)
lpm1Dr = Dropout(0.5, seed=42)(lpm1)
lpm2 = Dense(1024, activation='relu', kernel_initializer='glorot_normal', bias_initializer='glorot_normal',
            kernel_regularizer=regularizers.l2(0.01))(lpm1Dr)
lpm2Dr = Dropout(0.5, seed=42)(lpm2)
lpm3 = Dense(5, activation=None, kernel_initializer='glorot_normal', bias_initializer='glorot_normal')(lpm2Dr)
lpmS = Activation('softmax')(lpm3)
# Make into single network
vggConvSleep = Model(inputs=vggInput, outputs=lpmS)
# If conv layers should not be trained: 
# Maybe this should be excluded
for layer in vggConvSleep.layers[:2]:
    layer.trainable = False

# Optimizer
optimize = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

# Compile the model
vggConvSleep.compile(loss='categorical_crossentropy', optimizer=optimize, metrics=['accuracy'])

# Get model summary
vggConvSleep.summary()

## Define training mode

In [ ]:
training_mode = 'source'
# training_mode = 'target'
# training_mode = 'dann'

## Load data

In [ ]:
# Number of subjects for each group
num_subjects_physionet = 20
num_subjects_hospital = 17

# Load all data into memory
# Change data path before running !!
data_physionet = pickle.load(open('/home/jaskmo/Documents/DataCollection/sleep-edfx/PickleJar/dataOut.pkl','rb'))
data_hospital = pickle.load(open('/home/jaskmo/Documents/DataCollection/sleep-edfx/PickleJar/hDataOut.pkl', 'rb'))
random_perm_physionet = np.random.permutation(num_subjects_physionet)
random_perm_hospital = np.random.permutation(num_subjects_hospital)
idx_tmp_physionet = random_perm_physionet[range(num_subjects_physionet - 3)]
idx_test_physionet = random_perm_physionet[(num_subjects_physionet - 3):num_subjects_physionet]
idx_tmp_hospital = random_perm_hospital[range(num_subjects_hospital - 3)]
idx_test_hospital = random_perm_hospital[(num_subjects_hospital- 3) : num_subjects_hospital]
inputs_train_phys, targets_train_phys, inputs_val_phys, targets_val_phys, inputs_test_phys, targets_test_phys = get_data_complete(
    idx_tmp_physionet, idx_test_physionet, data_physionet, 'physionet')
inputs_train_hosp, targets_train_hosp, inputs_val_hosp, targets_val_hosp, inputs_test_hosp, targets_test_hosp = get_data_complete(
    idx_tmp_hospital, idx_test_hospital, data_hospital, 'hospital')

## Learning rate adaptation

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=5, min_lr=0.000001, verbose=1)
early_stop = EarlyStopping(monitor = 'val_loss', patience=7)

## Fit model and save

In [ ]:
# Fit model
# Change Save paths before running !!
if training_mode == 'source': # Training on source data from physionet
    csv_logger = CSVLogger('models/sourceModel.log')
    vggConvSleep.fit(x=inputs_train_phys, y=targets_train_phys, validation_data=(inputs_val_phys, targets_val_phys), 
                     batch_size=50, epochs=80, verbose=1, callbacks=[early_stop, reduce_lr, csv_logger], shuffle=True)
elif training_mode == 'target': # Training on target data from hospital
    csv_logger = CSVLogger('models/targetModel.log')
    vggConvSleep.fit(x=inputs_train_hosp, y=targets_train_hosp, validation_data=(inputs_val_hosp, targets_val_hosp), 
                     batch_size=40, epochs=80, verbose=1, callbacks=[early_stop, reduce_lr, csv_logger], shuffle=True)
# save model
vggConvSleep.save(filepath='/home/jaskmo/Documents/programering/02456-deep-learning/Project/RES/kerasSource_dec21.h5')

## Evaluate model on both source and target

In [ ]:
#loss, metric = vggConvSleep.evaluate(x=inputs_test_phys, y=targets_test_phys, batch_size=50)
target_names = ['W','N1','N2','N3','R']

if training_mode == 'source': # Training on source data from physionet
    targets_test_int = [np.where(r == 1)[0][0] for r in targets_test_phys]
    y_pred = vggConvSleep.predict(inputs_test_phys)
    y_pred2 = np.argmax(y_pred, axis = 1)
    # Test accuracy:
    acc = accuracy_score(targets_test_int, y_pred2)
    print('Accuracy in this domain = ', acc)
        
    conf_mat = confusion_matrix(targets_test_int, y_pred2)
    print(conf_mat)
    # Per class metrics
    class_report = classification_report(targets_test_int, y_pred2, target_names=target_names)
    print(class_report)
    
    # Evaluate error on target data
    _, metric = vggConvSleep.evaluate(x=inputs_test_hosp, y=targets_test_hosp, batch_size=50)
    print('Accuracy on other domain = ', metric)
    
    
elif training_mode == 'target': # Training on target data from hospital
    # Convert from onehot
    targets_test_int = [np.where(r == 1)[0][0] for r in targets_test_hosp]
    y_pred = vggConvSleep.predict(inputs_test_hosp)
    y_pred2 = np.argmax(y_pred, axis = 1)
    # Test accuracy:
    acc = accuracy_score(targets_test_int, y_pred2)
    print('Accuracy in this domain = ', acc)
    # Confusion matrix for target
    conf_mat = confusion_matrix(targets_test_int, y_pred2)
    print(conf_mat)
    # Per class metrics
    class_report = classification_report(targets_test_int, y_pred2, target_names=target_names)
    print(class_report)
    
    # Evaluate error on source data
    _, metric = vggConvSleep.evaluate(x=inputs_test_phys, y=targets_test_phys, batch_size=50)
    print('Accuracy on other domain = ', metric)